In [2]:
%run imports.py

Succesfully imported libraries and modules



In [3]:
# loading patients
p1 = load_patient(1)
p2 = load_patient(2)
p3 = load_patient(3)
p4 = load_patient(4)
p5 = load_patient(5)
p6 = load_patient(6)

Patient directory:  D:\LAB\AlexPAC\data\Patient1
Patient directory:  D:\LAB\AlexPAC\data\Patient2
Patient directory:  D:\LAB\AlexPAC\data\Patient3
Patient directory:  D:\LAB\AlexPAC\data\Patient4
Patient directory:  D:\LAB\AlexPAC\data\Patient5
Patient directory:  D:\LAB\AlexPAC\data\Patient6


In [4]:
cross_placements = ["L4-3A", "L4-3B", "L4-3C", "L2A-3A", "L2B-3B", "L2C-3C", "L1-2A", "L1-2B", "L1-2C", 
                    "R4-3A", "R4-3B", "R4-3C", "R2A-3A", "R2B-3B", "R2C-3C", "R1-2A", "R1-2B", "R1-2C"]

for patient in [p1, p2, p3, p4, p5, p6]:
    patient.cross_placements = cross_placements

In [69]:
columns = ['patient', 'condition', 'phase', 'amplitude', 'beta_power', 'hfo_power', 'pac_power_p2', 'pac_power_p09', 'is_cross_pac']
data = pd.DataFrame(columns=columns)

for patient in [p1, p2, p3, p4, p5, p6]:
    
    conditions = [c for c in patient.conditions if 'Rest 180sec' in c]
    for condition in tqdm(conditions, total=len(conditions)):
        day, ldopa, movement = retrieve_condition_name(condition)
        for pp in patient.placements:
            
            # get beta power for phase-giving oscillation
            lfp = patient.lfp[condition][pp]
            beta_power = get_beta_power(lfp)

            for pa in patient.placements:
                if not pac_exists(patient, condition, pp, pa):
                    continue
                
                # if irrelevant cross-PAC (not all conditions will be available)
                if (pp != pa) and (pp not in patient.cross_placements or pa not in patient.cross_placements):
                    continue
                
                # get HFO power for amplitude-giving oscillation
                lfp = patient.lfp[condition][pa]
                hfo_power = get_hfo_power(lfp)

                # Get PAC power
                pac = patient.pac[condition][pp][pa]

                ## getting only significant PAC + leaving only "clustered" entries
                sig_pac = get_sig_pac(pac, significance=0.01, mask_lonely=True)

                ## adjusting for different matrix shapes of cross-PACs
                if (pp != pa) & (pac.pac_matrix.shape == (19, 27)): 
                    pac_power_p2 = compute_matrix_norm(sig_pac, p=2)
                    pac_power_p09 = compute_matrix_norm(sig_pac, p=0.9)
                else:
                    beta10 = np.argmin(np.abs(pac.beta_freqs - 10))
                    beta36 = np.argmin(np.abs(pac.beta_freqs - 36))
                    hfo140 = np.argmin(np.abs(pac.hfo_freqs - 140))
                    pac_power_p2 = compute_matrix_norm(sig_pac[hfo140:, beta10:beta36 + 1], p=2)
                    pac_power_p09 = compute_matrix_norm(sig_pac[hfo140:, beta10:beta36 + 1], p=0.9)
                
                entry = {'patient': patient.name, 
                      'condition': condition,
                      'phase': pp, 
                      'amplitude': pa, 
                      'beta_power': beta_power, 
                      'hfo_power': hfo_power, 
                      'pac_power_p2': pac_power_p2, 
                      'pac_power_p09': pac_power_p09,
                      'is_cross_pac': int(pp != pa)
                     }

                data = data.append(entry, ignore_index = True)
                
data['ldopa'] = data['condition'].apply(lambda s: s.split(' ')[1])
    
data.to_csv('beta_hfo_pac_powers_REST.csv', sep=',')
data.head()

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

,patient,condition,phase,amplitude,beta_power,hfo_power,pac_power_p2,pac_power_p09,is_cross_pac,ldopa
0,Patient1,1Day ON Rest 180sec,R1-2A,R1-2A,0.471097,0.001178,0.0,0.0,0,ON
1,Patient1,1Day ON Rest 180sec,R1-2A,R4-3B,0.471097,0.001514,0.0,0.0,1,ON
2,Patient1,1Day ON Rest 180sec,R1-2A,R1-2B,0.471097,0.000648,0.0,0.0,1,ON
3,Patient1,1Day ON Rest 180sec,R1-2A,R4-3C,0.471097,0.001486,0.0,0.0,1,ON
4,Patient1,1Day ON Rest 180sec,R1-2A,R2B-3B,0.471097,0.000949,0.0,0.0,1,ON


In [55]:
df = pd.read_csv('beta_hfo_pac_powers_REST.csv', sep=',', index_col=0)

In [56]:
df.groupby('condition').count()

,patient,phase,amplitude,beta_power,hfo_power,pac_power_p2,pac_power_p09,is_cross_pac,ldopa
condition,,,,,,,,,
1Day OFF Rest 180sec,1044,1044,1044,1044,1044,1044,1044,1044,1044
1Day ON Rest 180sec,1044,1044,1044,1044,1044,1044,1044,1044,1044
5Day OFF Rest 180sec,870,870,870,870,870,870,870,870,870
5Day ON Rest 180sec,870,870,870,870,870,870,870,870,870


In [57]:
df.max()

patient                     Patient6
condition        5Day ON Rest 180sec
phase                          R4-3C
amplitude                      R4-3C
beta_power                   1.47023
hfo_power                 0.00296581
pac_power_p2              0.00292404
pac_power_p09              0.0251258
is_cross_pac                       1
ldopa                             ON
dtype: object

In [58]:
df.mean()

beta_power       0.616537
hfo_power        0.000535
pac_power_p2     0.000114
pac_power_p09    0.000582
is_cross_pac     0.827586
dtype: float64

In [60]:
df = pd.read_csv('beta_hfo_pac_powers_REST.csv', sep=',', index_col=0)

for patient in [p1, p2, p3, p4, p5, p6]:
    print(patient.name)
    conditions = list(df[df.patient == patient.name].groupby('condition').groups.keys())
    print(conditions)

    for phase in patient.sorted_placements:
        for amplitude in patient.sorted_placements:
            n_cond = []
            for condition in conditions:
                mask = (df.patient == patient.name) & \
                       (df.phase == phase) & \
                       (df.amplitude == amplitude) & \
                       (df.condition == condition)
                n_cond.append(len(df[mask]))
            if (n_cond[1] != n_cond[0]):
                print(f"{phase}->{amplitude}: {n_cond}")


Patient1
['1Day OFF Rest 180sec', '1Day ON Rest 180sec', '5Day OFF Rest 180sec', '5Day ON Rest 180sec']
Patient2
['1Day OFF Rest 180sec', '1Day ON Rest 180sec', '5Day OFF Rest 180sec', '5Day ON Rest 180sec']
Patient3
['1Day OFF Rest 180sec', '1Day ON Rest 180sec', '5Day OFF Rest 180sec', '5Day ON Rest 180sec']
Patient4
['1Day OFF Rest 180sec', '1Day ON Rest 180sec', '5Day OFF Rest 180sec', '5Day ON Rest 180sec']
Patient5
['1Day OFF Rest 180sec', '1Day ON Rest 180sec', '5Day OFF Rest 180sec', '5Day ON Rest 180sec']
Patient6
['1Day OFF Rest 180sec', '1Day ON Rest 180sec']


## Exact same method for DF but OFF vs ON

In [107]:
columns = list({'patient': None, 
                 'phase': None, 
                 'amplitude': None,
                 'day': None,
                 'is_cross_pac': None,
                 'beta_power_OFF': None, 
                 'beta_power_ON': None, 
                 'hfo_power_OFF': None, 
                 'hfo_power_ON': None, 
                 'pac_power_OFF': None, 
                 'pac_power_ON': None}.keys())

print(columns)

conditions_rest = ["1Day OFF Rest 180 sec", "1Day ON Rest 180sec", "5Day OFF Rest 180 sec", "5Day ON Rest 180sec"]

data = pd.DataFrame(columns=columns)
patients = [p1, p2, p3, p4, p5, p6]

for patient in patients:             
    for pp in tqdm(patient.sorted_placements, total = len(patient.sorted_placements)):
        for pa in patient.sorted_placements:
            for day in ["1Day", "5Day"]:
                
                entry = {'patient': patient.name, 
                         'phase': pp, 
                         'amplitude': pa,
                         'day': day,
                         'is_cross_pac': int(pp != pa),
                         'beta_power_OFF': None, 
                         'beta_power_ON': None, 
                         'hfo_power_OFF': None, 
                         'hfo_power_ON': None, 
                         'pac_power_OFF': None, 
                         'pac_power_ON': None}
                
                for ldopa in ["OFF", "ON"]:
                    condition = create_condition_name(day, ldopa, "Rest 180sec")
                    if not pac_exists(patient, condition, pp, pa):
                        #print("PAC doesn't exist")
                        continue
                    #print(f"{pp}->{pa} : {condition}")

                    # if irrelevant cross-PAC (not all conditions will be available)
                    if (pp != pa) and (pp not in patient.cross_placements or pa not in patient.cross_placements):
                        #print("Irrelevant PAC")
                        continue

                    # get beta power
                    lfp = patient.lfp[condition][pp]
                    beta_power = get_beta_power(lfp)
                    entry[f'beta_power_{ldopa}'] = beta_power

                    # get HFO power
                    lfp = patient.lfp[condition][pa]
                    hfo_power = get_hfo_power(lfp)
                    entry[f'hfo_power_{ldopa}'] = hfo_power

                    # Get PAC power
                    pac = patient.pac[condition][pp][pa]

                    ## getting only significant PAC + leaving only "clustered" entries
                    sig_pac = get_sig_pac(pac, significance=0.01, mask_lonely=True)

                    ## adjusting for different matrix shapes of cross-PACs
                    if (pp != pa) & (pac.pac_matrix.shape == (19, 27)): 
                        pac_power = compute_matrix_norm(sig_pac, p=2)
                    else:
                        beta10 = np.argmin(np.abs(pac.beta_freqs - 10))
                        beta36 = np.argmin(np.abs(pac.beta_freqs - 36))
                        hfo140 = np.argmin(np.abs(pac.hfo_freqs - 140))
                        pac_power = compute_matrix_norm(sig_pac[hfo140:, beta10:beta36 + 1], p=2)

                    entry[f'pac_power_{ldopa}'] = pac_power

                data = data.append(entry, ignore_index = True)

data = data.dropna()
data.to_csv('df_OFFvsON_new.csv', sep=',')
data.head(10)

['patient', 'phase', 'amplitude', 'day', 'is_cross_pac', 'beta_power_OFF', 'beta_power_ON', 'hfo_power_OFF', 'hfo_power_ON', 'pac_power_OFF', 'pac_power_ON']


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

,patient,phase,amplitude,day,is_cross_pac,beta_power_OFF,beta_power_ON,hfo_power_OFF,hfo_power_ON,pac_power_OFF,pac_power_ON
0,Patient1,R1-2C,R1-2C,1Day,0,0.64257,0.532455,0.000345155,0.00069344,0.000128785,0
1,Patient1,R1-2C,R1-2C,5Day,0,1.24167,0.364734,0.000259356,0.000389991,0.000303676,0
2,Patient1,R1-2C,R1-2B,1Day,1,0.64257,0.532455,0.000353132,0.000647707,0,0
3,Patient1,R1-2C,R1-2B,5Day,1,1.24167,0.364734,0.000254761,0.000468457,0,0.000168708
4,Patient1,R1-2C,R1-2A,1Day,1,0.64257,0.532455,0.000369624,0.00117828,0,0
5,Patient1,R1-2C,R1-2A,5Day,1,1.24167,0.364734,0.000296961,0.000419489,0,0
12,Patient1,R1-2C,R2C-3C,1Day,1,0.64257,0.532455,0.000276046,0.00156788,0,0
13,Patient1,R1-2C,R2C-3C,5Day,1,1.24167,0.364734,0.00029424,0.000475318,0.000654982,0
14,Patient1,R1-2C,R2B-3B,1Day,1,0.64257,0.532455,0.00022158,0.000948799,0,0
15,Patient1,R1-2C,R2B-3B,5Day,1,1.24167,0.364734,0.000358144,0.000421765,0.000453783,0


In [108]:
mask = (data.phase == "R1-2A") & (data.amplitude == "R1-2A")
data[mask]

,patient,phase,amplitude,day,is_cross_pac,beta_power_OFF,beta_power_ON,hfo_power_OFF,hfo_power_ON,pac_power_OFF,pac_power_ON
124,Patient1,R1-2A,R1-2A,1Day,0,0.634618,0.471097,0.000369624,0.00117828,0,0
125,Patient1,R1-2A,R1-2A,5Day,0,1.11959,0.431273,0.000296961,0.000419489,0,0
1924,Patient2,R1-2A,R1-2A,1Day,0,0.601738,0.518295,0.00107497,0.00125883,0,0.000119795
1925,Patient2,R1-2A,R1-2A,5Day,0,1.09474,0.316624,0.000197583,0.000450343,0.00018116,0.000195001
3724,Patient3,R1-2A,R1-2A,1Day,0,0.617667,0.198719,0.000251453,0.000532125,0,0
3725,Patient3,R1-2A,R1-2A,5Day,0,0.825979,0.465585,8.90049e-05,0.000281295,0,0
5524,Patient4,R1-2A,R1-2A,1Day,0,0,0.418915,0.000412908,0.000460963,0,0
5525,Patient4,R1-2A,R1-2A,5Day,0,0.384816,0.300573,0.000405762,0.000367308,0,0.000370002
7324,Patient5,R1-2A,R1-2A,1Day,0,0.386309,0,0.000418239,0.000608536,0,0
7325,Patient5,R1-2A,R1-2A,5Day,0,0.700801,0,0.000282149,0.000251136,0,0.000110368


In [109]:
df_rest.head()

,patient,condition,phase,amplitude,beta_power,hfo_power,pac_power_p2,pac_power_p09,is_cross_pac,ldopa
0,Patient1,1Day ON Rest 180sec,R1-2A,R1-2A,0.471097,0.001178,0.0,0.0,0,ON
1,Patient1,1Day ON Rest 180sec,R1-2A,R4-3B,0.471097,0.001514,0.0,0.0,1,ON
2,Patient1,1Day ON Rest 180sec,R1-2A,R1-2B,0.471097,0.000648,0.0,0.0,1,ON
3,Patient1,1Day ON Rest 180sec,R1-2A,R4-3C,0.471097,0.001486,0.0,0.0,1,ON
4,Patient1,1Day ON Rest 180sec,R1-2A,R2B-3B,0.471097,0.000949,0.0,0.0,1,ON


In [110]:
len(data)

1914

In [111]:
data.max()

patient             Patient6
phase                  R4-3C
amplitude              R4-3C
day                     5Day
is_cross_pac               1
beta_power_OFF       1.47023
beta_power_ON       0.927353
hfo_power_OFF     0.00296581
hfo_power_ON      0.00296473
pac_power_OFF     0.00292404
pac_power_ON      0.00268502
dtype: object

In [112]:
data[data.is_cross_pac == 0].max()

patient             Patient6
phase                  R4-3C
amplitude              R4-3C
day                     5Day
is_cross_pac               0
beta_power_OFF       1.47023
beta_power_ON       0.927353
hfo_power_OFF     0.00296581
hfo_power_ON      0.00296473
pac_power_OFF      0.0020074
pac_power_ON      0.00230447
dtype: object

In [91]:
df_rest = pd.read_csv("beta_hfo_pac_powers_REST.csv", sep=',', index_col=0)

In [113]:
df_rest.max()

patient                     Patient6
condition        5Day ON Rest 180sec
phase                          R4-3C
amplitude                      R4-3C
beta_power                   1.47023
hfo_power                 0.00296581
pac_power_p2              0.00292404
pac_power_p09              0.0251258
is_cross_pac                       1
ldopa                             ON
dtype: object

In [102]:
df_rest[df_rest.is_cross_pac == 0].max()

patient                     Patient6
condition        5Day ON Rest 180sec
phase                          R4-3C
amplitude                      R4-3C
beta_power                   1.47023
hfo_power                 0.00296581
pac_power_p2              0.00230447
pac_power_p09              0.0203888
is_cross_pac                       0
ldopa                             ON
dtype: object

In [114]:
data.groupby('phase').count()

,patient,amplitude,day,is_cross_pac,beta_power_OFF,beta_power_ON,hfo_power_OFF,hfo_power_ON,pac_power_OFF,pac_power_ON
phase,,,,,,,,,,
L1-2A,99,99,99,99,99,99,99,99,99,99
L1-2B,99,99,99,99,99,99,99,99,99,99
L1-2C,99,99,99,99,99,99,99,99,99,99
L2A-2C,11,11,11,11,11,11,11,11,11,11
L2A-3A,99,99,99,99,99,99,99,99,99,99
L2B-2A,11,11,11,11,11,11,11,11,11,11
L2B-3B,99,99,99,99,99,99,99,99,99,99
L2C-2B,11,11,11,11,11,11,11,11,11,11
L2C-3C,99,99,99,99,99,99,99,99,99,99


In [73]:

data.to_csv('df_OFFvsON_new.csv', sep=',')
data.head()

,patient,condition,phase,amplitude,beta_power,hfo_power,pac_power_p2,pac_power_p09,is_cross_pac,ldopa,beta_power_OFF,beta_power_ON,day,hfo_power_OFF,hfo_power_ON,pac_power_OFF,pac_power_ON


### DataFrame with direct OFF vs ON comparison

In [65]:
verbose = False


columns = ['patient', 
         'phase',
         'amplitude', 
         'is_cross_pac', 
         'day',
         'beta_power_OFF',
         'beta_power_ON',
         'hfo_power_OFF',
         'hfo_power_ON',
         'pac_power_p2_OFF',
         'pac_power_p2_ON',
         'pac_power_p09_OFF', 
         'pac_power_p09_ON']

df_offvson = pd.DataFrame(columns=columns)

for patient_name in tqdm(["Patient" + str(i) for i in range(1, 7)], total=6):
    if verbose: print(patient_name)
    conditions = list(df[df.patient == patient_name].groupby('condition').groups.keys())
    print(conditions)
    placements = p1.sorted_placements
    for pp in placements:
        for pa in placements:
            if verbose: print(f"{pp}->{pa}")
            for condition_off in [c for c in conditions if 'OFF' in c]:


                day, ldopa, movement = retrieve_condition_name(condition_off)

                mask_off = (df.patient == patient.name) & \
                           (df.phase == pp) & \
                           (df.amplitude == pa) & \
                           (df.condition == condition_off)

                condition_on = create_condition_name(day, 'ON', movement)
                
                if verbose: print(condition_off + " || " + condition_on)
                
                mask_on = (df.patient == patient.name) & \
                          (df.phase == pp) & \
                          (df.amplitude == pa) & \
                          (df.condition == condition_on)
                
                if verbose:
                    if np.sum(mask_off) != 0:
                        print("Found OFF entry")
                        print(df[mask_off])
                        
                    if np.sum(mask_on) != 0:
                        print("Found ON entry")
                        print(df[mask_on])
                
                if np.sum(mask_off) * np.sum(mask_on) == 0:
                    continue

                df_off = df[mask_off]
                #print(df_off)
                #print(df_off)
                df_on = df[mask_on]
                #print(df_on)
                
                #print(df_on)
                

                entry = {'patient': patient_name, 
                         'phase': pp, 
                         'amplitude': pa,
                         'day': day,
                         'is_cross_pac': df_off['is_cross_pac'].values[0],
                         'beta_power_OFF': df_off['beta_power'].values[0], 
                         'beta_power_ON': df_on['beta_power'].values[0], 
                         'hfo_power_OFF': df_off['hfo_power'].values[0], 
                         'hfo_power_ON': df_on['hfo_power'].values[0], 
                         'pac_power_p2_OFF': df_off['pac_power_p2'].values[0], 
                         'pac_power_p2_ON': df_on['pac_power_p2'].values[0], 
                         'pac_power_p09_OFF': df_off['pac_power_p09'].values[0], 
                         'pac_power_p09_ON': df_on['pac_power_p09'].values[0]}
                
                if verbose:
                    if day == '5Day':
                        print("Found 5 Day")
                        print(entry)
                        

                df_offvson = df_offvson.append(entry, ignore_index=True)
df_offvson.to_csv("df_OFFvsON_Rest.csv", sep=',')                
df_offvson.head()

  0%|          | 0/6 [00:00<?, ?it/s]

['1Day OFF Rest 180sec', '1Day ON Rest 180sec', '5Day OFF Rest 180sec', '5Day ON Rest 180sec']
['1Day OFF Rest 180sec', '1Day ON Rest 180sec', '5Day OFF Rest 180sec', '5Day ON Rest 180sec']
['1Day OFF Rest 180sec', '1Day ON Rest 180sec', '5Day OFF Rest 180sec', '5Day ON Rest 180sec']
['1Day OFF Rest 180sec', '1Day ON Rest 180sec', '5Day OFF Rest 180sec', '5Day ON Rest 180sec']
['1Day OFF Rest 180sec', '1Day ON Rest 180sec', '5Day OFF Rest 180sec', '5Day ON Rest 180sec']
['1Day OFF Rest 180sec', '1Day ON Rest 180sec']


,patient,phase,amplitude,is_cross_pac,day,beta_power_OFF,beta_power_ON,hfo_power_OFF,hfo_power_ON,pac_power_p2_OFF,pac_power_p2_ON,pac_power_p09_OFF,pac_power_p09_ON
0,Patient1,R1-2C,R1-2C,0,1Day,0.433831,0.0,0.000098,0.000793,0.000141,0.0,0.000274,0.0
1,Patient1,R1-2C,R1-2B,1,1Day,0.433831,0.0,0.000099,0.000351,0.000000,0.0,0.000000,0.0
2,Patient1,R1-2C,R1-2A,1,1Day,0.433831,0.0,0.000098,0.000513,0.000000,0.0,0.000000,0.0
3,Patient1,R1-2C,R2C-3C,1,1Day,0.433831,0.0,0.000117,0.001262,0.000000,0.0,0.000000,0.0
4,Patient1,R1-2C,R2B-3B,1,1Day,0.433831,0.0,0.000076,0.001215,0.000000,0.0,0.000000,0.0


In [66]:
df_offvson.groupby('day').count()

,patient,phase,amplitude,is_cross_pac,beta_power_OFF,beta_power_ON,hfo_power_OFF,hfo_power_ON,pac_power_p2_OFF,pac_power_p2_ON,pac_power_p09_OFF,pac_power_p09_ON
day,,,,,,,,,,,,
1Day,1044,1044,1044,1044,1044,1044,1044,1044,1044,1044,1044,1044


In [67]:
df_offvson.mean()

is_cross_pac         0.827586
beta_power_OFF       0.642996
beta_power_ON        0.519413
hfo_power_OFF        0.000148
hfo_power_ON         0.000923
pac_power_p2_OFF     0.000041
pac_power_p2_ON      0.000036
pac_power_p09_OFF    0.000095
pac_power_p09_ON     0.000087
dtype: float64

In [68]:
df_offvson.max()

patient                 Patient6
phase                      R4-3C
amplitude                  R4-3C
is_cross_pac                   1
day                         1Day
beta_power_OFF            1.0043
beta_power_ON           0.818852
hfo_power_OFF         0.00033884
hfo_power_ON          0.00215205
pac_power_p2_OFF     0.000286428
pac_power_p2_ON      0.000474686
pac_power_p09_OFF    0.000931055
pac_power_p09_ON      0.00131002
dtype: object

In [49]:
data = pd.read_csv('beta_hfo_pac_powers_REST.csv', sep=',', index_col=0)

len(data)

3828

In [50]:
df_offvson = pd.read_csv('df_OFFvsON_Rest.csv', sep=',', index_col=0)

len(df_offvson)

1914

In [51]:
df_offvson.groupby('phase').count()['patient']

phase
L1-2A     99
L1-2B     99
L1-2C     99
L2A-2C    11
L2A-3A    99
L2B-2A    11
L2B-3B    99
L2C-2B    11
L2C-3C    99
L3A-3C    11
L3B-3A    11
L3C-3B    11
L4-3A     99
L4-3B     99
L4-3C     99
R1-2A     99
R1-2B     99
R1-2C     99
R2A-2C    11
R2A-3A    99
R2B-2A    11
R2B-3B    99
R2C-2B    11
R2C-3C    99
R3A-3C    11
R3B-3A    11
R3C-3B    11
R4-3A     99
R4-3B     99
R4-3C     99
Name: patient, dtype: int64

In [29]:
data.groupby('phase').count()['patient']

phase
L1-2A     198
L1-2B     198
L1-2C     198
L2A-2C     22
L2A-3A    198
L2B-2A     22
L2B-3B    198
L2C-2B     22
L2C-3C    198
L3A-3C     22
L3B-3A     22
L3C-3B     22
L4-3A     198
L4-3B     198
L4-3C     198
R1-2A     198
R1-2B     198
R1-2C     198
R2A-2C     22
R2A-3A    198
R2B-2A     22
R2B-3B    198
R2C-2B     22
R2C-3C    198
R3A-3C     22
R3B-3A     22
R3C-3B     22
R4-3A     198
R4-3B     198
R4-3C     198
Name: patient, dtype: int64

### df OFF vs ON (1 patient)

In [31]:
patient_name = 'Patient1'

df = pd.read_csv('beta_hfo_pac_powers_REST.csv', sep=',', index_col=0)

columns = ['patient', 
         'phase',
         'amplitude', 
         'is_cross_pac', 
         'day',
         'beta_power_OFF',
         'beta_power_ON',
         'hfo_power_OFF',
         'hfo_power_ON',
         'pac_power_p2_OFF',
         'pac_power_p2_ON',
         'pac_power_p09_OFF', 
         'pac_power_p09_ON']

df_offvson = pd.DataFrame(columns=columns)

 
conditions = list(df[df.patient == patient_name].groupby('condition').groups.keys())
print(conditions)
placements = p1.placements
counter_0 = 0
counter_1 = 0
for pp in placements:
    for pa in placements:
        for condition_off in [c for c in conditions if 'OFF' in c]:
            
            day, ldopa, movement = retrieve_condition_name(condition_off)
            if day == '5Day':
                counter_0 += 1

            mask_off = (df.patient == patient.name) & \
                       (df.phase == pp) & \
                       (df.amplitude == pa) & \
                       (df.condition == condition_off)

            condition_on = create_condition_name(day, 'ON', movement)
            mask_on = (df.patient == patient.name) & \
                      (df.phase == pp) & \
                      (df.amplitude == pa) & \
                      (df.condition == condition_on)

            df_off = df[mask_off]
            #print(df_off)
            #print(df_off)
            df_on = df[mask_on]
            #print(df_on)
            if len(df_off) == 0 or len(df_on) == 0:
                continue
            #print(df_on)
            if day == '5Day':
                counter_1 += 1
            entry = {'patient': patient_name, 
                     'phase': pp, 
                     'amplitude': pa,
                     'is_cross_pac': df_off['is_cross_pac'].values[0],
                     'day': day, 
                     'beta_power_OFF': df_off['beta_power'].values[0], 
                     'beta_power_ON': df_on['beta_power'].values[0], 
                     'hfo_power_OFF': df_off['hfo_power'].values[0], 
                     'hfo_power_ON': df_on['hfo_power'].values[0], 
                     'pac_power_p2_OFF': df_off['pac_power_p2'].values[0], 
                     'pac_power_p2_ON': df_on['pac_power_p2'].values[0], 
                     'pac_power_p09_OFF': df_off['pac_power_p09'].values[0], 
                     'pac_power_p09_ON': df_on['pac_power_p09'].values[0]}
            #print(entry)

            df_offvson = df_offvson.append(entry, ignore_index=True)
            
df_offvson.head()

['1Day OFF Rest 180sec', '1Day ON Rest 180sec', '5Day OFF Rest 180sec', '5Day ON Rest 180sec']


,patient,phase,amplitude,is_cross_pac,day,beta_power_OFF,beta_power_ON,hfo_power_OFF,hfo_power_ON,pac_power_p2_OFF,pac_power_p2_ON,pac_power_p09_OFF,pac_power_p09_ON
0,Patient1,R1-2A,R1-2A,0,1Day,0.54401,0.304835,0.000098,0.000513,0.000128,0.00000,0.000191,0.000000
1,Patient1,R1-2A,R4-3B,1,1Day,0.54401,0.304835,0.000131,0.000878,0.000000,0.00000,0.000000,0.000000
2,Patient1,R1-2A,R1-2B,1,1Day,0.54401,0.304835,0.000099,0.000351,0.000000,0.00000,0.000000,0.000000
3,Patient1,R1-2A,R4-3C,1,1Day,0.54401,0.304835,0.000096,0.001504,0.000139,0.00012,0.000262,0.000231
4,Patient1,R1-2A,R2B-3B,1,1Day,0.54401,0.304835,0.000076,0.001215,0.000173,0.00000,0.000396,0.000000


In [22]:
df_offvson

,patient,phase,amplitude,is_cross_pac,day,beta_power_OFF,beta_power_ON,hfo_power_OFF,hfo_power_ON,pac_power_p2_OFF,pac_power_p2_ON,pac_power_p09_OFF,pac_power_p09_ON
0,Patient1,R1-2A,R1-2A,0,1Day,0.54401,0.304835,0.000098,0.000513,0.000128,0.000000,0.000191,0.000000
1,Patient1,R1-2A,R4-3B,1,1Day,0.54401,0.304835,0.000131,0.000878,0.000000,0.000000,0.000000,0.000000
2,Patient1,R1-2A,R1-2B,1,1Day,0.54401,0.304835,0.000099,0.000351,0.000000,0.000000,0.000000,0.000000
3,Patient1,R1-2A,R4-3C,1,1Day,0.54401,0.304835,0.000096,0.001504,0.000139,0.000120,0.000262,0.000231
4,Patient1,R1-2A,R2B-3B,1,1Day,0.54401,0.304835,0.000076,0.001215,0.000173,0.000000,0.000396,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Patient1,L2A-3A,L4-3A,1,1Day,1.00430,0.818852,0.000270,0.000636,0.000000,0.000243,0.000000,0.001037
170,Patient1,L2A-3A,L4-3B,1,1Day,1.00430,0.818852,0.000061,0.000284,0.000000,0.000000,0.000000,0.000000
171,Patient1,L2A-3A,L1-2B,1,1Day,1.00430,0.818852,0.000240,0.000654,0.000000,0.000142,0.000000,0.000253
172,Patient1,L2A-3A,L1-2C,1,1Day,1.00430,0.818852,0.000145,0.000625,0.000000,0.000475,0.000000,0.000886


In [24]:
for patient_name in ["Patient" + str(i) for i in range(1, 7)]:
    print(len(df[df.patient == patient_name]))


696
696
696
696
696
348


In [ ]:
df_offvson

In [17]:
df_offvson.groupby('phase').count()['patient']

phase
L1-2A     54
L1-2B     54
L1-2C     54
L2A-2C     6
L2A-3A    54
L2B-2A     6
L2B-3B    54
L2C-2B     6
L2C-3C    54
L3A-3C     6
L3B-3A     6
L3C-3B     6
L4-3A     54
L4-3B     54
L4-3C     54
R1-2A     54
R1-2B     54
R1-2C     54
R2A-2C     6
R2A-3A    54
R2B-2A     6
R2B-3B    54
R2C-2B     6
R2C-3C    54
R3A-3C     6
R3B-3A     6
R3C-3B     6
R4-3A     54
R4-3B     54
R4-3C     54
Name: patient, dtype: int64

In [74]:
counter_0

900

In [75]:
counter_1

174

In [55]:
df_offvson.head()

,patient,phase,amplitude,is_cross_pac,day,beta_power_OFF,beta_power_ON,hfo_power_OFF,hfo_power_ON,pac_power_p2_OFF,pac_power_p2_ON,pac_power_p09_OFF,pac_power_p09_ON


{'1Day': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]}

In [43]:
condition_off = '5Day OFF Rest 180sec'
patient = p1

day, ldopa, movement = retrieve_condition_name(condition_off)

mask_off = (df.patient == patient.name) & \
           (df.phase == pp) & \
           (df.amplitude == pa) & \
           (df.condition == condition_off)

condition_on = create_condition_name(day, 'ON', movement)
mask_on = (df.patient == patient.name) & \
          (df.phase == pp) & \
          (df.amplitude == pa) & \
          (df.condition == condition_on)

df_off = df[mask_off]
print(df_off)
df_on = df[mask_on]
print(df_on)
entry = {'patient': patient_name, 
         'phase': pp, 
         'amplitude': pa,
         'is_cross_pac': df_off['is_cross_pac'].values[0],
         'day': day, 
         'beta_power_OFF': df_off['beta_power'].values[0], 
         'beta_power_ON': df_on['beta_power'].values[0], 
         'hfo_power_OFF': df_off['hfo_power'].values[0], 
         'hfo_power_ON': df_on['hfo_power'].values[0], 
         'pac_power_p2_OFF': df_off['pac_power_p2'].values[0], 
         'pac_power_p2_ON': df_on['pac_power_p2'].values[0], 
         'pac_power_p09_OFF': df_off['pac_power_p09'].values[0], 
         'pac_power_p09_ON': df_on['pac_power_p09'].values[0]}

print(entry)

#df_offvson.append(entry, ignore_index=True)

      patient             condition   phase amplitude  beta_power  hfo_power  \
521  Patient1  5Day OFF Rest 180sec  L2A-3A    L2A-3A    0.609522   0.000388   

     pac_power_p2  pac_power_p09  is_cross_pac ldopa  
521           0.0            0.0             0   OFF  
      patient            condition   phase amplitude  beta_power  hfo_power  \
347  Patient1  5Day ON Rest 180sec  L2A-3A    L2A-3A         0.0   0.000435   

     pac_power_p2  pac_power_p09  is_cross_pac ldopa  
347       0.00019       0.000492             0    ON  
{'patient': 'Patient6', 'phase': 'L2A-3A', 'amplitude': 'L2A-3A', 'is_cross_pac': 0, 'day': '5Day', 'beta_power_OFF': 0.6095223051142864, 'beta_power_ON': 0.0, 'hfo_power_OFF': 0.0003878216693605135, 'hfo_power_ON': 0.00043476025740810375, 'pac_power_p2_OFF': 0.0, 'pac_power_p2_ON': 0.00019013034418809862, 'pac_power_p09_OFF': 0.0, 'pac_power_p09_ON': 0.0004920927285596219}


In [47]:
df_on.size

0

In [24]:
df_offvson.head()

,patient,phase,amplitude,is_cross_pac,day,beta_power_OFF,beta_power_ON,hfo_power_OFF,hfo_power_ON,pac_power_p2_OFF,pac_power_p2_ON,pac_power_p09_OFF,pac_power_p09_ON
